In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Academic/Acad2014.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#Combining title and text
df["Text"] = df["Title"] + df["Abstract"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Abstract'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,StoreId,documentType,year,Text
0,1650982901,"Journal , Peer Reviewed Journal , Journal Art...",2014,A randomized controlled trial of mental health...
1,1650985265,"Journal , Peer Reviewed Journal , Journal Art...",2014,Staff and service users’ views on a ‘Consent f...
2,1650983695,"Journal , Peer Reviewed Journal , Journal Art...",2014,Early risk factors for adult bipolar disorder ...
3,1647030557,"Journal , Peer Reviewed Journal , Journal Art...",2014,Comparison of psychiatric disability on the He...
4,1647030599,"Journal , Peer Reviewed Journal , Journal Art...",2014,A validation study of the Hospital Anxiety and...


In [6]:
# Convert to list
data=df.Text.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['randomized', 'controlled', 'trial', 'of', 'mental', 'health', 'interventions', 'for', 'survivors', 'of', 'systematic', 'violence', 'in', 'kurdistan', 'northern', 'iraqbackground', 'experiencing', 'systematic', 'violence', 'and', 'trauma', 'increases', 'the', 'risk', 'of', 'poor', 'mental', 'health', 'outcomes', 'few', 'interventions', 'for', 'these', 'types', 'of', 'exposures', 'have', 'been', 'evaluated', 'in', 'low', 'resource', 'contexts', 'the', 'objective', 'of', 'this', 'randomized', 'controlled', 'trial', 'was', 'to', 'assess', 'the', 'effectiveness', 'of', 'two', 'interventions', 'behavioral', 'activation', 'treatment', 'for', 'depression', 'batd', 'and', 'cognitive', 'processing', 'therapy', 'cpt', 'in', 'reducing', 'depression', 'symptoms', 'using', 'locally', 'adapted', 'and', 'validated', 'version', 'of', 'the', 'hopkins', 'symptom', 'checklist', 'and', 'dysfunction', 'measured', 'with', 'locally', 'developed', 'scale', 'secondary', 'outcomes', 'included', 'posttraumatic

In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['randomized_controlled_trial', 'of', 'mental', 'health', 'interventions', 'for', 'survivors', 'of', 'systematic', 'violence', 'in', 'kurdistan', 'northern', 'iraqbackground', 'experiencing', 'systematic', 'violence', 'and', 'trauma', 'increases', 'the', 'risk', 'of', 'poor', 'mental', 'health', 'outcomes', 'few', 'interventions', 'for', 'these', 'types', 'of', 'exposures', 'have_been', 'evaluated', 'in', 'low', 'resource', 'contexts', 'the', 'objective', 'of', 'this', 'randomized_controlled_trial', 'was', 'to', 'assess', 'the', 'effectiveness', 'of', 'two', 'interventions', 'behavioral', 'activation', 'treatment', 'for', 'depression', 'batd', 'and', 'cognitive', 'processing', 'therapy', 'cpt', 'in', 'reducing', 'depression', 'symptoms', 'using', 'locally', 'adapted', 'and', 'validated', 'version', 'of', 'the', 'hopkins', 'symptom', 'checklist', 'and', 'dysfunction', 'measured', 'with', 'locally', 'developed', 'scale', 'secondary', 'outcomes', 'included', 'posttraumatic_stress', 'anxie

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=56,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['randomized_controlle', 'trial', 'mental', 'health', 'intervention', 'survivor', 'systematic', 'violence', 'experience', 'systematic', 'violence', 'trauma', 'increase', 'risk', 'poor', 'mental', 'health', 'outcome', 'intervention', 'type', 'exposure', 'evaluate', 'low', 'resource', 'contexts', 'objective', 'randomized_controlle', 'trial', 'assess', 'effectiveness', 'intervention', 'behavioral', 'activation', 'treatment', 'depression', 'cognitive', 'processing', 'therapy', 'reduce', 'depression', 'symptom', 'use', 'locally', 'adapt', 'validate', 'version', 'dysfunction', 'measure', 'locally', 'develop', 'scale', 'secondary', 'outcome', 'include', 'anxiety', 'traumatic', 'grief', 'symptom', 'method', 'community', 'mental', 'health', 'worker', 'work', 'rural', 'health', 'clinic', 'randomly_assigne', 'train', 'intervention', 'community', 'mental', 'health', 'worker', 'conduct', 'baseline', 'assessment', 'enrol', 'survivor', 'systematic', 'violence', 'base', 'severity', 'depression', 'sym

In [15]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(16,
  '0.000*"indication" + 0.000*"head" + 0.000*"sustainability" + '
  '0.000*"reluctant" + 0.000*"kind" + 0.000*"infancy" + 0.000*"underway" + '
  '0.000*"rest" + 0.000*"digital" + 0.000*"being"'),
 (35,
  '0.000*"service" + 0.000*"kind" + 0.000*"underway" + 0.000*"technology" + '
  '0.000*"sustainability" + 0.000*"reluctant" + 0.000*"accompany" + '
  '0.000*"infancy" + 0.000*"attractive" + 0.000*"esbi"'),
 (31,
  '0.000*"indication" + 0.000*"head" + 0.000*"sustainability" + '
  '0.000*"reluctant" + 0.000*"kind" + 0.000*"infancy" + 0.000*"underway" + '
  '0.000*"rest" + 0.000*"digital" + 0.000*"being"'),
 (1,
  '0.000*"indication" + 0.000*"head" + 0.000*"sustainability" + '
  '0.000*"reluctant" + 0.000*"kind" + 0.000*"infancy" + 0.000*"underway" + '
  '0.000*"rest" + 0.000*"digital" + 0.000*"being"'),
 (54,
  '0.000*"mental" + 0.000*"disorder" + 0.000*"health" + 0.000*"self" + '
  '0.000*"symptom" + 0.000*"efficacy" + 0.000*"factor" + 0.000*"depression" + '
  '0.000*"effect" + 0.00

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -12.725220374714448


In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.36150730245500157


In [18]:
num_topics = 56
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [19]:
for i in range(0,56):
    print(topic_words[i])

['healthcare', 'list', 'disorder', 'health', 'subthreshold', 'symptom', 'earthquake', 'study', 'mental', 'risk']
['indication', 'head', 'sustainability', 'reluctant', 'kind', 'infancy', 'underway', 'rest', 'digital', 'being']
['marker', 'single', 'mental', 'health', 'estimate', 'population', 'burden', 'high', 'report', 'poor']
['patient', 'late', 'rate', 'high', 'depression', 'severity', 'level', 'functional', 'measure', 'symptom']
['profile', 'subject', 'combination', 'suffer', 'effect', 'animal', 'neuroimage', 'previously', 'drug', 'investigation']
['fear', 'disorder', 'associate', 'youth', 'anxiety', 'survey', 'extinction', 'non', 'study', 'include']
['subthreshold', 'syndrome', 'recurrent', 'disease', 'morbidity', 'epidemiological', 'typically', 'considerably', 'threshold', 'public']
['individual', 'pattern', 'client', 'ptsd', 'event', 'present', 'trauma', 'trajectory', 'service', 'traumatic']
['counselling', 'success', 'immediately', 'far', 'linkage', 'ptc', 'hazardous', 'cmd', 'a

In [20]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})
df.head()

,Most_freq_words,Source,Topic_ID,Year


In [21]:
df['Most_freq_words']=topic_words

In [22]:
df = df.assign(Year='2014')
df = df.assign(Source='Academic')
df.shape

(56, 4)

In [23]:
ls=[]
for i in range(0,56):
    ls.append(i)
df['Topic_ID']=ls

In [24]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[healthcare, list, disorder, health, subthresh...",Academic,0,2014
1,"[indication, head, sustainability, reluctant, ...",Academic,1,2014
2,"[marker, single, mental, health, estimate, pop...",Academic,2,2014
3,"[patient, late, rate, high, depression, severi...",Academic,3,2014
4,"[profile, subject, combination, suffer, effect...",Academic,4,2014


In [25]:
df.to_csv("topic_words_j2014.csv")